# Demonstrate Classification

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from CONFIG import PREPROCESSED_DATA_PATH
from knn import KNN
from gensim.models import Word2Vec
import gensim.downloader
from classifier import Classifier
from word2vec import Word2VecModel

In [3]:
df_data = pd.read_parquet(PREPROCESSED_DATA_PATH)

# Get Title Embeddings

In [3]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

In [6]:
def get_average_word_embeddings(title):
    word_embeddings = [word2vec[x] for x in title.split() if x in word2vec]
    if len(word_embeddings) == 0:
        word_embeddings = np.zeros((1, 300))
    word_embeddings = np.array(word_embeddings)
    return np.mean(word_embeddings, axis=-2)

In [4]:
test_subset = df_data[:100]
train_subset = df_data[100:10000]

# Classify Using Word Embeddings

In [7]:
training_vectors = [get_average_word_embeddings(x) for x in train_subset["title"]]
training_classes = train_subset["categories"]

NameError: name 'word2vec' is not defined

In [4]:
tokens = np.array([x.split() for x in train_subset["title"]])
w2v_custom = Word2VecModel()
w2v_custom.fit(tokens, epochs=64, batch_size=1024)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


Epoch 1/64
9/9 - 0s - loss: 0.6924
Epoch 2/64
9/9 - 0s - loss: 0.6908
Epoch 3/64
9/9 - 0s - loss: 0.6885
Epoch 4/64
9/9 - 0s - loss: 0.6865
Epoch 5/64
9/9 - 0s - loss: 0.6834
Epoch 6/64
9/9 - 0s - loss: 0.6795
Epoch 7/64
9/9 - 0s - loss: 0.6738
Epoch 8/64
9/9 - 0s - loss: 0.6690
Epoch 9/64
9/9 - 0s - loss: 0.6651
Epoch 10/64
9/9 - 0s - loss: 0.6576
Epoch 11/64
9/9 - 0s - loss: 0.6515
Epoch 12/64
9/9 - 0s - loss: 0.6478
Epoch 13/64
9/9 - 0s - loss: 0.6394
Epoch 14/64
9/9 - 0s - loss: 0.6338
Epoch 15/64
9/9 - 0s - loss: 0.6262
Epoch 16/64
9/9 - 0s - loss: 0.6197
Epoch 17/64
9/9 - 0s - loss: 0.6170
Epoch 18/64
9/9 - 0s - loss: 0.6100
Epoch 19/64
9/9 - 0s - loss: 0.6061
Epoch 20/64
9/9 - 0s - loss: 0.6014
Epoch 21/64
9/9 - 0s - loss: 0.5914
Epoch 22/64
9/9 - 0s - loss: 0.5872
Epoch 23/64
9/9 - 0s - loss: 0.5896
Epoch 24/64
9/9 - 0s - loss: 0.5835
Epoch 25/64
9/9 - 0s - loss: 0.5773
Epoch 26/64
9/9 - 0s - loss: 0.5810
Epoch 27/64
9/9 - 0s - loss: 0.5745
Epoch 28/64
9/9 - 0s - loss: 0.5690
E

In [5]:
w2v_custom.save("test")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: test/assets


In [4]:
m = Word2VecModel()
m.load("test")

In [5]:
c = Classifier("test")

In [6]:
c.fit(train_subset["title"], train_subset["categories"])

/home/ruchtia/git/word2vec/word2vec/classifier.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  word_embeddings = np.array(word_embeddings)


AxisError: axis -2 is out of bounds for array of dimension 1

In [ ]:
c.predict(train_subset["title"])

# K-Fold Validation

In [ ]:
from sklearn.model_selection import KFold
import sklearn

In [ ]:
c = Classifier("trained_word2vec")

In [ ]:
df_data = df_data.sample(frac=1).reset_index(drop=True)
classes = np.array(df_data["categories"])
classes = classes[:10000]
titles = np.array(df_data["title"])
titles = titles[:10000]

In [ ]:
kf = KFold(n_splits=10)
i = 0
for train_index, test_index in kf.split(titles):
    print("FOLD ", i)
    X_train, X_test = titles[train_index], titles[test_index]
    y_train, y_test = classes[train_index], classes[test_index]
    c.fit(X_train, y_train)
    
    predictions = c.predict(X_test)
    f_score = sklearn.metrics.f1_score(y_test, predictions, average='macro')
    precision = sklearn.metrics.precision_score(y_test, predictions, average='macro')
    recall = sklearn.metrics.recall_score(y_test, predictions, average='macro')
    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    print("F Score:", f_score, "Precision", precision, "Recall", recall, "Accuracy", acc)

In [11]:
import pandas as pd
import numpy as np
from classifier import Classifier

c = Classifier("trained_word2vec")

# Load Data
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")


In [12]:

# Fit KNN (word2vec already fit)
c.fit(np.array(df_train["title"].astype(str))[:10000], np.array(df_train["categories"].astype(str))[:10000])

# Simple exampe
print("Ground Truth", np.array(df_test["categories"])[:10])
print("Predicted", c.predict(np.array(df_test["title"]))[:10])

Ground Truth ['books' 'toysgames' 'sportsoutdoors' 'cellphonesaccessories'
 'clothingshoesjewelry' 'clothingshoesjewelry' 'clothingshoesjewelry'
 'homekitchen' 'automotive' 'toolshomeimprovement']
Predicted ['books' 'homekitchen' 'automotive' 'cellphonesaccessories'
 'clothingshoesjewelry' 'clothingshoesjewelry' 'clothingshoesjewelry'
 'clothingshoesjewelry' 'automotive' 'electronics']
